In [1]:
# For Important MariaDB Connector
import subprocess
import sys
def install_dep_mariadb():
    """
    This is for installing maria db lib using terminal
    """
    command = ['sudo', 'apt-get', 'install', 'libmariadb-dev']
    try:
        print(f"Executing command: {' '.join(command)}")
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        print(result.stdout)
    except Exception as e:
        print(f"I Don't know some error ig {e}")
install_dep_mariadb()

Executing command: sudo apt-get install libmariadb-dev
Reading package lists...
Building dependency tree...
Reading state information...
The following packages were automatically installed and are no longer required:
  libaom-dev libarmadillo-dev libarpack2-dev libblosc-dev libcfitsio-dev
  libdav1d-dev libde265-dev libfreexl-dev libfyba-dev libgeos-dev
  libgeotiff-dev libgif-dev libhdf4-alt-dev libheif-dev libjson-c-dev
  libkml-dev libkmlconvenience1 libkmlregionator1 libkmlxsd1 liblz4-dev
  libminizip-dev libnetcdf-dev libodbccr2 libogdi-dev libpoppler-dev
  libpoppler-private-dev libpq-dev libproj-dev libqhull-dev libqhull8.0
  libqhullcpp8.0 librttopo-dev libspatialite-dev libsqlite3-dev libsuperlu-dev
  liburiparser-dev libwebp-dev libx265-dev libxerces-c-dev unixodbc-dev
Use 'sudo apt autoremove' to remove them.
The following additional packages will be installed:
  libmariadb3 mariadb-common
The following packages will be REMOVED:
  default-libmysqlclient-dev libgdal-dev libmy

In [2]:
!pip install pandas sqlalchemy scikit-learn mariadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.1/111.1 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mariadb: filename=mariadb-1.1.13-cp311-cp311-linux_x86_64.whl size=201571 sha256=2d11c3e381c140b5c9e1d58e7a56ef5a53b12d4baf6dbe703f0358410b8767d9
  Stored in directory: /root/.cache/pip/wheels/42/30/bb/6bb56b4072e3388e5788edd7fdfa16a81dcca702175da20684
Successfully built mariadb


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sqlalchemy import create_engine
import mariadb
from kaggle_secrets import UserSecretsClient
import joblib

In [4]:
user_secrets = UserSecretsClient()

DB_HOST = user_secrets.get_secret("DB_HOST")
DB_NAME = user_secrets.get_secret("DB_NAME")
DB_PASSWORD = user_secrets.get_secret("DB_PASSWORD")
DB_PORT = user_secrets.get_secret("DB_PORT")
DB_USER = user_secrets.get_secret("DB_USER")

# This query is for subsetting and mixing
sql_query = """
SELECT
    a.id, a.name, a.short_description, a.positive_reviews, a.negative_reviews, a.achievements_count,
    GROUP_CONCAT(DISTINCT g.name SEPARATOR ' ') AS genres,
    GROUP_CONCAT(DISTINCT c.name SEPARATOR ' ') AS categories,
    GROUP_CONCAT(DISTINCT t.name SEPARATOR ' ') AS tags
FROM
    apps a
LEFT JOIN app_genres ag ON a.id = ag.app_id
LEFT JOIN genres g ON ag.genre_id = g.id
LEFT JOIN app_categories ac ON a.id = ac.app_id
LEFT JOIN categories c ON ac.category_id = c.id
LEFT JOIN app_tags atg ON a.id = atg.app_id
LEFT JOIN tags t ON atg.tag_id = t.id
WHERE
    a.type = 'game' AND a.positive_reviews + a.negative_reviews > 100
GROUP BY
    a.id;
"""
engine = create_engine(
    f"mariadb+mariadbconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)
df = pd.read_sql(sql_query, engine)

In [5]:
print(f"Data loaded successfully. Found {len(df)} games.")
print("Starting feature engineering...")
df.fillna('', inplace=True)
df.head()

Data loaded successfully. Found 20986 games.
Starting feature engineering...


,id,name,short_description,positive_reviews,negative_reviews,achievements_count,genres,categories,tags
0,10,Counter-Strike,Play the world's number 1 online action game. ...,243818,6427,0,Action,Color Alternatives Custom Volume Controls Fami...,1980s 1990's Action Assassin Classic Competiti...
1,20,Team Fortress Classic,One of the most popular online action games of...,7602,1136,0,Action,Custom Volume Controls Family Sharing Keyboard...,1990's Action Class-Based Classic Co-op Compet...
2,30,Day of Defeat,Enlist in an intense brand of Axis vs. Allied ...,6414,688,0,Action,Camera Comfort Color Alternatives Custom Volum...,Action Class-Based Classic Co-op Difficult Fir...
3,40,Deathmatch Classic,Enjoy fast-paced multiplayer gaming with Death...,2618,545,0,Action,Color Alternatives Custom Volume Controls Fami...,1990's Action Arena Shooter Classic Co-op Comp...
4,50,Half-Life: Opposing Force,Return to the Black Mesa Research Facility as ...,24363,1198,0,Action,Adjustable Difficulty Custom Volume Controls F...,1990's Action Adventure Aliens Atmospheric Cla...


In [6]:
for col in ['positive_reviews', 'negative_reviews', 
            'achievements_count']:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
for col in ['genres', 'categories', 'tags', 'short_description']:
    df[col] = df[col].fillna("")

In [7]:
def create_soup(x):
    return (x['short_description'] + ' ' + 
            x['genres'].replace('; ', ' ') + ' ' + 
            x['categories'].replace('; ', ' ') + ' ' + 
            x['tags'].replace('; ', ' ')).lower()
df['soup'] = df.apply(create_soup, axis=1)

In [8]:
tfidf = TfidfVectorizer(stop_words = "english")

In [9]:
tfidf_matrix = tfidf.fit_transform(df['soup'])

In [10]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [11]:
indices = pd.Series(df.index, index=df['name']).drop_duplicates()

In [12]:
number_of_game_recommendations = 20
def get_recommendations(title, cosine_sim=cosine_sim):
    """
    This function takes a game title and returns the top 5 most similar games.
    """
    if title not in indices:
        return f"Game '{title}' not found in the dataset."
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:number_of_game_recommendations]
    game_indices = [i[0] for i in sim_scores]
    return df['name'].iloc[game_indices]

In [13]:
game_title = 'Lightmatter'
recommendations = get_recommendations(game_title)

In [14]:
print(recommendations)

19833                                     Get To Work
3873                        Lakeview Cabin Collection
6957                                      Attempt[42]
14311                  Bright Paw: Definitive Edition
4389            Songs for a Hero - Definitive Edition
2817                                          The Way
7537                                          Nephise
11129                               The Last Campfire
5778                                         Event[0]
19175                                  SharpShooter3D
10407                                   The Last Cube
5435     ! That Bastard Is Trying To Steal Our Gold !
13922                                         Lab Rat
18266                     THE MULLER-POWELL PRINCIPLE
5504                                  The Eyes of Ara
3625                                          Armillo
7556                                         TARTARUS
14770                                   Gravity Field
629                        H